##  <i class="fa fa-spinner"></i> 装载自定义模块
把当前repo根目录加入 Python 搜索模块的路径列表，并求得根目录

In [ ]:
import os,sys

def find_repo_root():
    # 获取当前脚本的绝对路径
    try:
        # 如果当前运行环境是 Jupyter Notebook，使用当前工作目录
        script_path = os.path.abspath('.')
    except NameError:
        # 否则，使用 __file__ 变量
        script_path = os.path.abspath(__file__)
    # 判断当前脚本目录是否存在`.git`文件夹
    if os.path.isdir(os.path.join(script_path, '.git')):
        return script_path
    # 初始化存储结果的变量
    result = None
    # 循环求当前路径的父目录，直到找到`.git`文件夹
    while True:
        # 将当前路径的父目录赋值给当前路径
        script_path = os.path.dirname(script_path)
        # 判断当前路径是否存在`.git`文件夹
        if os.path.isdir(os.path.join(script_path, '.git')):
            # 如果存在，将当前路径存储在结果变量中
            result = script_path
        # 判断当前路径是否为根目录（即是否已经搜索到最外层）
        if script_path == '/':
            # 如果是，返回结果变量的值
            return result
        
repo_root_dir=find_repo_root()
sys.path.append(repo_root_dir)
print(repo_root_dir)
# 获取repo所在的根目录
root_dir = os.path.dirname(repo_root_dir)
# 获取当前环境名称与当前环境content路径
from func.env import detect_environment
env_cb = detect_environment()
env_name = env_cb['env_name']
content_path = env_cb['content_path']

## <i class="fa fa-plane-departure"></i> 学术加速
首先要进行学术加速，这有利于拉取资源，详情请看：https://www.autodl.com/docs/network_turbo/  

In [ ]:
import sys
sys.path.append('../') # 因为func与ipynb位于同一个目录下，所以要往上一层路径索引
from func.env import setProxy
cb=setProxy()
proxy=cb['proxy']
region=cb['region']

## <i class="fa fa-arrow-down"></i> 安装必要依赖

In [ ]:
import subprocess

libraries_to_install = [
    
]

tools_to_install = [
    
]

installed_libraries = subprocess.run(["pip", "freeze"], capture_output=True).stdout.decode().split("\n")

# 尝试安装所有未安装的库
for library in libraries_to_install:
    if library == "black[jupyter]" and any("black" in installed_library for installed_library in installed_libraries):
        print(f"【 {library} 】已经安装，跳过安装")
        continue
    elif not any(library in installed_library for installed_library in installed_libraries):
        !pip install $library
    else:
        print(f"【 {library} 】已经安装，跳过安装")
        
# 尝试安装所有未安装的工具
update_needed = False
for tool in tools_to_install:
    exit_code = subprocess.run(["dpkg", "-s", tool], capture_output=True).returncode
    if exit_code != 0:
        update_needed = True
        break
    else:
        print(f"【 {tool} 】已经安装，跳过安装")
if update_needed:
    !apt-get update
    for tool in tools_to_install:
        exit_code = subprocess.run(["dpkg", "-s", tool], capture_output=True).returncode
        if exit_code != 0:
            !apt-get install -y $tool
    


## <i class="fa fa-search"></i> 查找InvokeAI返回项目所在路径

In [ ]:
import os
from func.checksum import generate_checksum,find_folder

def find_dir(root_dir, target_name):
    paths = []
    for dirpath, dirnames, filenames in os.walk(root_dir):
        for dirname in dirnames:
            if dirname == target_name:
                path = os.path.join(dirpath, dirname)
                paths.append(path)
    paths.sort(key=len)
    if paths:
        return paths
    else:
        return None

# print(generate_checksum('/root/InvokeAI'))
# print(generate_checksum('/root/InvokeAI_runtime'))

# 通过校验和的方式查找InvokeAI的github repo文件夹
InvokeAI_dir = None
# InvokeAI_dir = find_folder(root_dir,'827a2072f6f785ae0abf58be160b69b0e165b9a28eb2a69bae426eb268f90127')

# 如果找不到，则采用低阶的办法，也就是文件夹名称完全匹配的版本进行查找（因为repo总是更新，会影响校验和的结果）
if not InvokeAI_dir:
    InvokeAI_dir=find_dir(root_dir, 'InvokeAI')[0]
    
print(InvokeAI_dir)
    
InvokeAI_runtime_dir = None
# InvokeAI_runtime_dir = find_folder(root_dir,'c7dd5c092639d9ac32ea5d1bbcaa9ec4ed9ade22d2c0c5621d395af5c1cda267')
if not InvokeAI_runtime_dir:
    InvokeAI_runtime_dir=find_dir(root_dir, 'InvokeAI_runtime')[0]

print(InvokeAI_runtime_dir)

## 查找文件
查找configure_invokeai.py和INITIAL_MODELS.yaml

In [ ]:
import os

def find_file(root_dir, file_name):
    paths = []
    for dirpath, dirnames, filenames in os.walk(root_dir):
        for filename in filenames:
            if filename == file_name:
                path = os.path.join(dirpath, filename)
                paths.append(path)
    paths.sort(key=len)
    if paths:
        return paths
    else:
        return None

config_invokeai_paths = find_file(InvokeAI_dir, 'configure_invokeai.py')

if config_invokeai_paths:
    print(config_invokeai_paths)
else:
    print("Not Found.")

init_models_yaml_paths = find_file(root_dir, 'INITIAL_MODELS.yaml')

if init_models_yaml_paths:
    print(init_models_yaml_paths)
else:
    print("Not Found.")



## 替换YAML文件
用selected_hf_models.yaml覆盖INITIAL_MODELS.yaml

In [ ]:
import yaml

with open('./temp/selected_hf_models.yaml', 'r') as t:
    target_yaml_data = yaml.safe_load(t)

for index,i in enumerate(init_models_yaml_paths):
    filename=os.path.basename(i)
    temp_filepath = os.path.join('./temp',filename + f'_{index}')
    # 先进行备份
    !cp $i $temp_filepath
    # 再把文件覆盖
    !cp ./temp/selected_hf_models.yaml $i
    
    with open(i, 'r') as file:
        yaml_data = yaml.safe_load(file)
    if target_yaml_data == yaml_data:
        print(f'{i}已经替换成功')


## 下载模型

In [ ]:
base_args = [
    proxy,'&&',
    'source activate','&&',
    'conda activate invokeai','&&',
    f'cd {InvokeAI_dir}','&&'
]

run_configure_invokeai_args = base_args[:] + [f'python scripts/configure_invokeai.py -y --root_dir {InvokeAI_runtime_dir}']
run_configure_invokeai_cmd = (" ").join(run_configure_invokeai_args)

!$run_configure_invokeai_cmd

# 还原YAML文件
如果不还原，会影响InvokeAI的repo进行git pull，这样项目就不能顺利更新了

In [ ]:
for index,i in enumerate(init_models_yaml_paths):
    filename=os.path.basename(i)
    temp_filepath = os.path.join('./temp',filename + f'_{index}')
    !cp $temp_filepath $i && rm $temp_filepath